<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Data%20Mining%20Module%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##EDA

In [1]:
!pip install mlxtend -qqq

In [ ]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns
import mlxtend.preprocessing
import numpy
import datetime as dt
!pip install matplotlib


**check out the columns in the dataset in order to determine what columns to load**

In [4]:
base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'

# List of CSV file paths on GitHub
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv','sales_2024-03-07_2024-04-01.csv',
]

# Function to get columns from a single CSV file
def get_columns_from_csv(file_path):
    full_url = f'{base_url}{file_path}'
    # Read just the first row to get the columns
    df = pd.read_csv(full_url, nrows=1)
    return df.columns.tolist()

# Check columns in each dataset
all_columns = set()
for file_path in file_paths:
    columns = get_columns_from_csv(file_path)
    all_columns.update(columns)

# print all_columns and decide which ones to load
print(all_columns)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'variant_sku', 'product_type', 'order_name', 'month', 'customer_cohort_quarter', 'customer_type', 'net_sales', 'customer_cohort_month', 'units_per_transaction', 'customer_cohort_week', 'customer_id', 'product_price', 'shipping_postal_code', 'shipping_region', 'financial_status', 'total_cost', 'product_title', 'customer_email', 'total_sales', 'day'}


**load only the columns needed**

In [5]:
import pandas as pd

base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv','sales_2024-03-07_2024-04-01.csv',
]

columns_to_load = [
     'customer_type','customer_cohort_month',
    'order_name', 'month', 'units_per_transaction',
    'product_price', 'shipping_region','variant_sku','financial_status'
]

dataframes_list = []

for file_path in file_paths:
    full_url = f'{base_url}{file_path}'
    # First, check which columns exist in the CSV
    available_columns = pd.read_csv(full_url, nrows=0).columns.tolist()
    # Determine the intersection of available columns and columns to load
    columns_to_read = [column for column in columns_to_load if column in available_columns]
    df = pd.read_csv(full_url, usecols=columns_to_read)
    dataframes_list.append(df)

BBORDERS = pd.concat(dataframes_list, ignore_index=True)
BBORDERS.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction
0,2023-01,partially_refunded,#658025,Returning,SQ1726938,287.93,Texas,2021-07,1
1,2023-01,refunded,#667324,Returning,NaN,0.00,Florida,2022-07,0
2,2023-01,refunded,#667324,Returning,SQ9178530,269.94,Florida,2022-07,1
3,2023-01,refunded,#655100,Returning,SQ8862610,559.99,Indiana,2021-12,0
4,2023-01,refunded,#655100,Returning,NaN,0.00,Indiana,2021-12,0


**Need to check how a recurrent customers (not suscribiers) buying one time order shows in the report.**

In [8]:
#Filtered only orders paid representing sucessfull transactions,
#first time Customers representing New suscriptions.
#dropping missing values from variant_sku representing coupons

BB_filtered= BBORDERS[
    (BBORDERS['financial_status'] == 'paid') &
    (BBORDERS['customer_type'] == 'First-time')
].dropna(subset=['variant_sku'])

BB_filtered


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction
362,2023-01,paid,#658392,First-time,SQ8862610,54.99,New York,2023-01,1
377,2023-01,paid,#670349,First-time,BBNYBGLA,69.99,Virginia,2023-01,1
393,2023-01,paid,#666714,First-time,BB648B,109.99,Pennsylvania,2023-01,1
430,2023-01,paid,#661459,First-time,BB062-25,49.99,Connecticut,2023-01,1
459,2023-01,paid,#661431,First-time,BB736-37,64.99,Texas,2023-01,1
...,...,...,...,...,...,...,...,...,...
497163,NaN,paid,#891911,First-time,BB719-50,99.99,Wisconsin,2024-04,1
497164,NaN,paid,#892251,First-time,BB660B,84.99,Arizona,2024-04,1
497167,NaN,paid,#892262,First-time,BB1151-29,69.99,Wisconsin,2024-04,1
497193,NaN,paid,#892276,First-time,SQ9178530,179.97,Nevada,2024-04,1


In [9]:
# Dictionary for state name to state code mapping
state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

BB_filtered['state_code'] = BB_filtered['shipping_region'].map(state_codes)


BB_filtered


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,First-time,SQ8862610,54.99,New York,2023-01,1,NY
377,2023-01,paid,#670349,First-time,BBNYBGLA,69.99,Virginia,2023-01,1,VA
393,2023-01,paid,#666714,First-time,BB648B,109.99,Pennsylvania,2023-01,1,PA
430,2023-01,paid,#661459,First-time,BB062-25,49.99,Connecticut,2023-01,1,CT
459,2023-01,paid,#661431,First-time,BB736-37,64.99,Texas,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...
497163,NaN,paid,#891911,First-time,BB719-50,99.99,Wisconsin,2024-04,1,WI
497164,NaN,paid,#892251,First-time,BB660B,84.99,Arizona,2024-04,1,AZ
497167,NaN,paid,#892262,First-time,BB1151-29,69.99,Wisconsin,2024-04,1,WI
497193,NaN,paid,#892276,First-time,SQ9178530,179.97,Nevada,2024-04,1,NV


In [10]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Create a ProfileReport object
profile = ProfileReport(BB_filtered, title="Ecommerce", explorative=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# display report
profile

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
#checking for missing values
BB_filtered.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


month                    3562
financial_status            0
order_name                  0
customer_type               0
variant_sku                 0
product_price               0
shipping_region             1
customer_cohort_month       0
units_per_transaction       0
state_code                186
dtype: int64

I'm trying to preserve as many records as I possible can so I inspect the data looking for ways of silling missing values with the correct data

In [17]:
# Filter the DataFrame 'BB_filtered' to find rows where 'month' is NaN (missing)
missing_month_records = BB_filtered[BB_filtered['month'].isnull()]

# Display the records with missing 'month' values
missing_month_records


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code


In [15]:
# Grouping 'missing_month_records' by 'customer_cohort_month' and counting records in each group

missing_month_records.groupby('customer_cohort_month').size()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


customer_cohort_month
2024-03    3430
2024-04     132
dtype: int64

**After checking the orders in ShipStation, I can confirm the Month and Customer_cohort_month is the same for those orders.**

In [16]:
# Replace NaN values in 'month' column with values from 'customer_cohort_month'
BB_filtered['month'] = BB_filtered.apply(
    lambda row: row['customer_cohort_month'] if pd.isnull(row['month']) else row['month'],
    axis=1
)

BB_filtered

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,First-time,SQ8862610,54.99,New York,2023-01,1,NY
377,2023-01,paid,#670349,First-time,BBNYBGLA,69.99,Virginia,2023-01,1,VA
393,2023-01,paid,#666714,First-time,BB648B,109.99,Pennsylvania,2023-01,1,PA
430,2023-01,paid,#661459,First-time,BB062-25,49.99,Connecticut,2023-01,1,CT
459,2023-01,paid,#661431,First-time,BB736-37,64.99,Texas,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...
497163,2024-04,paid,#891911,First-time,BB719-50,99.99,Wisconsin,2024-04,1,WI
497164,2024-04,paid,#892251,First-time,BB660B,84.99,Arizona,2024-04,1,AZ
497167,2024-04,paid,#892262,First-time,BB1151-29,69.99,Wisconsin,2024-04,1,WI
497193,2024-04,paid,#892276,First-time,SQ9178530,179.97,Nevada,2024-04,1,NV


In [25]:
# Find all rows in the DataFrame where any column has NaN values
nan_records = BB_filtered[BB_filtered.isna().any(axis=1)]
nan_records


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
421635,2024-01,paid,#852808,First-time,BB719-50,0.0,Ma,2024-01,1,NaN
444484,2024-02,paid,#875516,First-time,BB694-18,0.0,NaN,2024-02,1,NaN


In [24]:
# Use the .loc method to target the 'state_code' column for rows where
# the 'shipping_region' is 'District Of Columbia'.
# This method allows us to select specific parts of the DataFrame based on a condition.
BB_filtered.loc[
   BB_filtered['shipping_region'] == 'District Of Columbia',  # Condition to filter rows
    'state_code'  # Column to apply the change
] = BB_filtered.loc[
    BB_filtered['shipping_region'] == 'District Of Columbia',  # Condition to filter rows again for consistency
    'state_code'  # Column to apply the .fillna() method
].fillna('DC')  # The .fillna() method replaces NaN values with 'DC'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
# Find all rows in the DataFrame where any column has NaN values
nan_records = BB_filtered[BB_filtered.isna().any(axis=1)]
nan_records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
421635,2024-01,paid,#852808,First-time,BB719-50,0.0,Ma,2024-01,1,NaN
444484,2024-02,paid,#875516,First-time,BB694-18,0.0,NaN,2024-02,1,NaN


In [30]:
# Filter the DataFrame for orders with a price of 0.0
orders_with_zero_price = BB_filtered[BB_filtered['product_price'] == 0.0]

# Count the number of such orders
number_of_zero_price_orders = orders_with_zero_price.shape[0]

# Display the count
print(f'Number of orders with a price of 0.0: {number_of_zero_price_orders}')

# Display the records with a price of 0.0
orders_with_zero_price


Number of orders with a price of 0.0: 4732


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
5434,2023-01,paid,#671997,First-time,BB395-23,0.0,Minnesota,2023-01,1,MN
8781,2023-01,paid,#660821,First-time,SQ1726938,0.0,Missouri,2023-01,1,MO
11404,2023-01,paid,#662029,First-time,SQ5173256,0.0,California,2023-01,1,CA
11819,2023-01,paid,#655548,First-time,BB062-25,0.0,Kentucky,2023-01,1,KY
19681,2023-01,paid,#655738,First-time,BB062-25,0.0,Kentucky,2023-01,1,KY
...,...,...,...,...,...,...,...,...,...,...
496827,2024-04,paid,#891923,First-time,SQ1726938,0.0,Wisconsin,2024-04,1,WI
496907,2024-03,paid,#890649,First-time,BB831-60,0.0,Illinois,2024-03,0,IL
497033,2024-04,paid,#892252,First-time,SQ1726938,0.0,Missouri,2024-04,1,MO
497087,2024-04,paid,#891481,First-time,BB125-29,0.0,New York,2024-04,1,NY


In [39]:
# Run in a Jupyter notebook cell to clone the repository
!git clone https://github.com/Requenamar3/Data-Mining.git

# Then change the directory
!cd Data-Mining


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Cloning into 'Data-Mining'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 232 (delta 89), reused 87 (delta 43), pack-reused 63
Receiving objects: 100% (232/232), 232.22 MiB | 27.57 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Updating files: 100% (44/44), done.


In [37]:
# Export the DataFrame 'orders_with_zero_price' to an Excel file without including the index
orders_with_zero_price.to_excel('orders_with_zero_price.xlsx', index=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

# Add the new file to your local Git repository
git add orders_with_zero_price.xlsx

# Commit the new file to your local Git repository
git commit -m "Add file with orders priced at 0.0"

# Push the commit to your remote repository on GitHub
git push origin your_branch_name


after checking in SS I can confirm that both orders are a test and need to be removed

In [ ]:
# Filter out rows with a 'product_price' of 0.0 from 'BB_filtered'
BB_filtered = BB_filtered.loc[BB_filtered['product_price'] != 0.0]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
BB_filtered

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_type,variant_sku,product_price,shipping_region,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,First-time,SQ8862610,54.99,New York,2023-01,1,NY
377,2023-01,paid,#670349,First-time,BBNYBGLA,69.99,Virginia,2023-01,1,VA
393,2023-01,paid,#666714,First-time,BB648B,109.99,Pennsylvania,2023-01,1,PA
430,2023-01,paid,#661459,First-time,BB062-25,49.99,Connecticut,2023-01,1,CT
459,2023-01,paid,#661431,First-time,BB736-37,64.99,Texas,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...
497163,NaN,paid,#891911,First-time,BB719-50,99.99,Wisconsin,2024-04,1,WI
497164,NaN,paid,#892251,First-time,BB660B,84.99,Arizona,2024-04,1,AZ
497167,NaN,paid,#892262,First-time,BB1151-29,69.99,Wisconsin,2024-04,1,WI
497193,NaN,paid,#892276,First-time,SQ9178530,179.97,Nevada,2024-04,1,NV


# Running the Apriori Algorithm and Finding Association Rules in Online Retail Datasets


**run the profiler to find duplicates and missing values**

##Part 1: Running the Apriori Algorithm on the Complete Online Retail Dataset

### 1.Run the Apriori Algorithm: Apply the Apriori algorithm to the full dataset. Set parameters to identify item sets appearing in at least 1% of transactions.


### 2.Analysis of Specific Item Set: Filter to the item set containing "10 COLOUR SPACEBOY PEN." Analyze its support value.

### 3.Item Set Length Analysis: Add a column for item set length. Focus on sets of length two with support between 0.02 and 0.021. Compare these to previous results.

### 4.Support Value Distribution: Plot the distribution of support values for the identified item sets

##Part 2: Finding the Association Rules on the Complete Online Retail Dataset

### 1.Fit Association Rule Model: Use the full dataset to generate association rules. Apply the confidence metric with a minimum threshold of 0.6.

### 2.Count and Compare Association Rules: Count the number of derived rules. Compare this with previous exercises.

### 3.Plot Confidence vs. Support: Create a plot to visualize the relationship between confidence and support for the rules.

### 4.Analyze Rule Metrics: Examine the distributions of confidence, lift, leverage, and conviction in your rules.

##Report writing



*   Insights gained from the Apriori algorithm and association rule mining.
*   How these insights can influence business decisions like store layout and pricing.

*   A comparison between the two datasets.
*   Reflections on how data partitioning and increasing data volume impact results.



